# Day 1: The Tyranny of the Rocket Equation

[Aoc Site](https://adventofcode.com/2019/day/1)

The Elves quickly load you into a spacecraft and prepare to launch.

## Part I

#### At the first Go / No Go poll, every Elf is Go until the Fuel Counter-Upper. They haven't determined the amount of fuel required yet.

Fuel required to launch a given module is based on its mass. ***Specifically, to find the fuel required for a module, take its mass, divide by three, round down, and subtract 2.*** For example:
- For a mass of 12, divide by 3 and round down to get 4, then subtract 2 to get 2.
- For a mass of 14, dividing by 3 and rounding down still yields 4, so the fuel required is also 2.
- For a mass of 1969, the fuel required is 654.
- For a mass of 100756, the fuel required is 33583.

The Fuel Counter-Upper needs to know the total fuel requirement. To find it, individually calculate the fuel needed for the mass of each module (your puzzle input), then add together all the fuel values.

***What is the sum of the fuel requirements for all of the modules on your spacecraft?***

To begin, [get your puzzle input](https://adventofcode.com/2019/day/1/input).

In [1]:
# Prep Notebook
import pandas as pd
import numpy as np

In [2]:
# Functionize calculation by mass

def fuel_calc(mass):
    return (mass // 3) - 2

In [3]:
# Functionize fuel-counter-upper

def fuel_counter_upper(df, mass_col='mass', fuel_col='fuel', test_id='p1'):
    df[fuel_col] = df[mass_col].apply(lambda x: fuel_calc(x))
    return df[fuel_col].sum()

In [4]:
# Functionize testing

def test_target_check(df, target_col='target', test_col='test', result_col='result'):
    df[result_col] = df[test_col] == df[target_col]
    return len(df[result_col]) == sum(df[result_col])

In [5]:
# Create test dataframe using test data

test_df = pd.DataFrame([12, 14, 1969, 100756], columns=['mass'])
test_df

,mass
0,12
1,14
2,1969
3,100756


In [6]:
# Test P1

mass_col = 'mass'
target_col1 = 'target_p1'
fuel_col = 'fuel_p1'
result_col1 = 'result_p1'

test_df[target_col1] = [2, 2, 654, 33583]
test_p1 = fuel_counter_upper(test_df, mass_col=mass_col, fuel_col=fuel_col)
result_p1 = test_target_check(test_df, target_col=target_col1, test_col=fuel_col, result_col=result_col1)
display(test_df)
print('Total:', test_p1, 'Result:', result_p1)

,mass,target_p1,fuel_p1,result_p1
0,12,2,2,True
1,14,2,2,True
2,1969,654,654,True
3,100756,33583,33583,True


Total: 34241 Result: True


In [7]:
# Load Data
mod_mass = pd.read_csv('../DataFiles/2019_01.txt', header=None, names=['mass'])
mod_mass.head()

,mass
0,148319
1,54894
2,105685
3,136247
4,133339


In [8]:
# Apply calculation to each module
mod_fuel = fuel_counter_upper(mod_mass, mass_col='mass', fuel_col='fuel', test_id=None)
mod_mass.head()

,mass,fuel
0,148319,49437
1,54894,18296
2,105685,35226
3,136247,45413
4,133339,44444


In [9]:
# Get sum of fuel
mod_fuel

3336985

### That's the right answer! You are *one gold star* closer to rescuing Santa.

## Part II

#### During the second Go / No Go poll, the Elf in charge of the Rocket Equation Double-Checker stops the launch sequence. 

Apparently, you forgot to include additional fuel for the fuel you just added.

Fuel itself requires fuel just like a module - take its mass, divide by three, round down, and subtract 2. However, that fuel also requires fuel, and that fuel requires fuel, and so on. Any mass that would require negative fuel should instead be treated as if it requires zero fuel; the remaining mass, if any, is instead handled by wishing really hard, which has no mass and is outside the scope of this calculation.

So, for each module mass, calculate its fuel and add it to the total. Then, treat the fuel amount you just calculated as the input mass and repeat the process, continuing until a fuel requirement is zero or negative. For example:
- A module of mass 14 requires 2 fuel. This fuel requires no further fuel (2 divided by 3 and rounded down is 0, which would call for a negative fuel), so the total fuel required is still just 2.
- At first, a module of mass 1969 requires 654 fuel. Then, this fuel requires 216 more fuel (654 / 3 - 2). 216 then requires 70 more fuel, which requires 21 fuel, which requires 5 fuel, which requires no further fuel. So, the total fuel required for a module of mass 1969 is 654 + 216 + 70 + 21 + 5 = 966.
- The fuel required by a module of mass 100756 and its fuel is: 33583 + 11192 + 3728 + 1240 + 411 + 135 + 43 + 12 + 2 = 50346.

What is the sum of the fuel requirements for all of the modules on your spacecraft when also taking into account the mass of the added fuel? (Calculate the fuel requirements for each module separately, then add them all up at the end.)

In [10]:
# Functionize fuel calculation

def fuel_recalc(fuel):
    refuel = fuel
    new_fuel = fuel
    while refuel > 8:
        refuel = max(0, fuel_calc(refuel))
        new_fuel += refuel
    return new_fuel

In [11]:
# Functionize fuel-topper-offer

def fuel_topper_offer(df, fuel_col='fuel', refuel_col='refuel'):
    df[refuel_col] = df[fuel_col].apply(lambda x: fuel_recalc(x))
    return df[refuel_col].sum()

In [12]:
# Test P1

target_col2 = 'target_p2'
fuel_col = 'fuel_p1'
refuel_col = 'refuel_p2'
result_col2 = 'result_p2'

test_df[target_col2] = [2, 2, 966, 50346]
test_p2 = fuel_topper_offer(test_df, fuel_col=fuel_col, refuel_col=refuel_col)
result_p2 = test_target_check(test_df, target_col=target_col2, test_col=refuel_col, result_col=result_col2)
display(test_df)
print('Total:', test_p2, 'Result:', result_p2)

,mass,target_p1,fuel_p1,result_p1,target_p2,refuel_p2,result_p2
0,12,2,2,True,2,2,True
1,14,2,2,True,2,2,True
2,1969,654,654,True,966,966,True
3,100756,33583,33583,True,50346,50346,True


Total: 51316 Result: True


In [13]:
# Apply calculation to each module

mod_refuel = fuel_topper_offer(mod_mass, fuel_col='fuel', refuel_col='refuel')
mod_mass.head()

,mass,fuel,refuel
0,148319,49437,74126
1,54894,18296,27417
2,105685,35226,52810
3,136247,45413,68089
4,133339,44444,66635


In [14]:
# Get sum of fuel
mod_refuel

5002611

### That's the right answer! You are *one gold star* closer to rescuing Santa.

You have completed Day 1!